In [38]:
import pandas as pd
pd.set_option("display.max_columns", None)

df_1 = pd.read_csv("../../../archive/with_rates_and_spec_gods.csv")
df_2 = pd.read_csv("../../../archive/2021_LoL_esports_match_data_from_SpinelessCoward_20210713.csv")

# That's right no rest for your boy the data organizer

In [39]:
df_1.rename(columns = {"blueChamps": "blueChampions", "redChamps": "redChampions", "redKills": "blueDeaths",
                      "blueHas62": "blueHasWukong", "redHas62": "redHasWukong", "blueHas111": "blueHasNautilus",
                      "redHas111": "redHasNautilus", "blueHas145": "blueHasKaiSa", "redHas145": "redHasKaisa",
                      "blueHas39": "blueHasIrelia", "redHas39": "redHasIrelia", "blueHas517": "blueHasSylas",
                      "redHas517": "redHasSylas"}, inplace = True)
# Lets make these column names be a little more readable

In [40]:
df_2.dropna(inplace = True)
# found some missing data, luckily there wasn't too much so I could justify dropping these rows

In [41]:
df_2["blueCSPerMin"], df_2["redCSPerMin"] = df_2["blueTotalCS"]/10, df_2["redTotalCS"]/10
df_2["blueGoldPerMin"], df_2["redGoldPerMin"] = df_2["blueTotalGold"]/10, df_2["redTotalGold"]/10
df_2 = df_2.drop(["blueTotalCS", "redTotalCS", "blueTotalGold", "redTotalGold"], axis = 1)

# making a couple new columns for the diamond players data set

In [42]:
import pickle

waifu = pickle.load(open("../../../Pickles/WaifuDictionary.pkl", "rb"))

# everyone loves pickles, lets use pickles to summon our potential waifu dictionary that can tell me all the leauge of
# legends player names based on id number

In [43]:
df_blueChampions, df_redChampions = df_1["blueChampions"], df_1["redChampions"]

# lets make some individual data frames so I don't accidentally mess things up

In [44]:
df_blueChampions = df_blueChampions.apply(lambda x: x.strip("[]"))
df_redChampions = df_redChampions.apply(lambda x: x.strip("[]"))

# formatting the champion lists

In [45]:
df_blueChampions = df_blueChampions.apply(lambda x: x.replace(" ", "").split(","))
df_redChampions = df_redChampions.apply(lambda x: x.replace(" ", "").split(","))

# Still formatting don't mind me

In [46]:
dict_i = {}
for i in range(1,156):
    number = waifu[i]["key"]
    name = waifu[i]["id"]
    dict_i[number] = name

# This for loop makes a dictionary that is just a champions name and id number

In [47]:
for i in range(len(df_blueChampions)):
    champs = []
    for x in range(5):
        champ = dict_i[df_blueChampions.iloc[i][x]]
        champs.append(champ)
                
    df_blueChampions.iloc[i] = champs
    
# using the above dictionary I replaced all the id numbers with champion names

In [48]:
for i in range(len(df_redChampions)):
    champs = []
    for x in range(5):
        champ = dict_i[df_redChampions.iloc[i][x]]
        champs.append(champ)
                
    df_redChampions.iloc[i] = champs
    
# gotta do it twice cause I made two different dataframes

In [49]:
df_1["blueChampions"], df_1["redChampions"] = df_blueChampions, df_redChampions

# putting the two dataframes back into the original dataframe they came from

In [72]:
# df_1.to_csv("../../../archive/with_rates_and_spec_gods_just_better.csv")
# saved to a csv file

In [52]:
df_2["blueChampions"] = df_2["blueChampions"].apply(lambda x: x.replace("'", ""))
df_2["redChampions"] = df_2["redChampions"].apply(lambda x: x.replace("'", ""))

# same format changes I did a couple cells up

In [53]:
blue_wins = df_2["blueChampions"].loc[df_2["blueWins"] == 1].apply(lambda x: x.strip("[]"))
red_wins = df_2["redChampions"].loc[df_2["blueWins"] == 0].apply(lambda x: x.strip("[]"))

# same format changes I did a couple cells up

In [54]:
blue_wins = blue_wins.apply(lambda x: x.strip(" ").replace('"', ''))
red_wins = red_wins.apply(lambda x: x.strip(" ").replace('"', ''))

# same format changes I did a couple cells up

In [55]:
blue_wins = blue_wins.apply(lambda x: x.replace(", ", ",").split(","))
red_wins = red_wins.apply(lambda x: x.replace(", ", ",").split(","))

# same format changes I did a couple cells up

In [56]:
wins = list(red_wins) + list(blue_wins)

# Now that I have a list of all the champions involved in wins I can make a champion wins dictionary

In [57]:
wins_per_champion = {}

for win in wins:
    for champ in win:
        wins_per_champion[champ] = wins_per_champion.get(champ, 0) + 1

# for loop that writes my dictionary to find total number of wins each champion has seen

In [58]:
blue_losses = df_2["blueChampions"].loc[df_2["blueWins"] == 0].apply(lambda x: x.strip("[]"))
red_losses = df_2["redChampions"].loc[df_2["blueWins"] == 1].apply(lambda x: x.strip("[]"))

blue_losses = blue_losses.apply(lambda x: x.replace(", ", ",").replace('"','').split(","))
red_losses = red_losses.apply(lambda x: x.replace(", ", ",").replace('"', '').split(","))

# same format changes but for the losses

In [59]:
losses = list(blue_losses) + list(red_losses)

# Now that I have a list of all the champions involved in losses I can make a champion losses dictionary

In [60]:
losses_per_champion = {}

for loss in losses:
    for champ in loss:
        losses_per_champion[champ] = losses_per_champion.get(champ, 0) + 1

# for loop that writes my dictionary to find total number of losses each champion has seen

In [61]:
champion_winrates = {}

for champ in set(list(wins_per_champion.keys()) + list(losses_per_champion.keys())):
    try:
        win = wins_per_champion[champ]
    except:
        win = 0
    try:
        loss = losses_per_champion[champ]
    except:
        loss = 0
    champion_winrates[champ] = win/(win + loss)
    
# building a champion winrate dictionary

In [62]:
champion1 = blue_losses + red_wins
champion2 = blue_wins + red_losses

# time to find champion pickrates so we need all the champions in a given game

In [63]:
champion_list = list(champion1) + list(champion2)

# we now have a list of all ten champions in a game so lets make a pickrate dictionary

In [64]:
champion_pickrates = {}

for game in champion_list:
    for champ in game:
        champion_pickrates[champ] = (champion_pickrates.get(champ, 0)+1)

# this gets us a dictionary of the number of times a champion was picked

In [65]:
for champ in champion_pickrates.keys():
    champion_pickrates[champ] = champion_pickrates[champ]/8278
# minor fix to give an actual pickrate

In [66]:
def average_pickrate(champions):
    total = 0
    for champ in champions:
        total += champion_pickrates[champ]
    return total/5

def average_winrate(champions):
    total = 0
    for champ in champions:
        total += champion_winrates[champ]
    return total/5

# making some functions to return an average winrate and pickrate

In [67]:
df_2["blueChampions"] = df_2["blueChampions"].map(lambda x: x.strip("[]").replace(", ", ",").replace('"','').split(","))
df_2["redChampions"] = df_2["redChampions"].map(lambda x: x.strip("[]").replace(", ", ",").replace('"','').split(","))

# same formatting issues as I had been having to do yet again. 

In [68]:
df_2["blueAvgPickrate"] = df_2["blueChampions"].map(average_pickrate)
df_2["redAvgPickrate"] = df_2["redChampions"].map(average_pickrate)

# making an average pickrate column for the pro dataframe

In [69]:
df_2["blueAvgWinrate"] = df_2["blueChampions"].map(average_winrate)
df_2["redAvgWinrate"] = df_2["redChampions"].map(average_winrate)

# making an average winrate column for the pro dataframe

In [71]:
# df_2.to_csv("../../../archive/this_took_way_too_much_effort_LoL_esports.csv")
# imported the updated pro dataframe to a csv